In [15]:
# Create circuit to test transpiler on
from qiskit import QuantumCircuit, transpile
#from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit.circuit.library import EfficientSU2
from qiskit.circuit import ParameterVector
from qiskit.result import QuasiDistribution

# Use Statevector object to calculate the ideal output
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram

# These are all for actual quantum computer 
from qiskit_ibm_runtime import Session
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit.providers.jobstatus import JobStatus 
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_provider import IBMProvider

from qiskit_algorithms import NumPyMinimumEigensolver, SamplingVQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit_finance.applications.optimization import PortfolioOptimization
from qiskit_finance.data_providers import RandomDataProvider
from qiskit_optimization.algorithms import MinimumEigenOptimizer

import numpy as np
import matplotlib.pyplot as plt
import datetime

## Setting up VQE

In [16]:
# Parameters from what we found was optimized in simulation
params = [ 2.64609588e-07, -4.01850864e-05,  2.66263480e-03,  4.48985492e-01,
  2.12317577e+00,  1.80258465e+00,  2.06128183e+00,  1.84302648e+00,
  1.75695393e+00,  1.61905146e+00,  9.96768151e-02,  9.90213915e-02,
  9.79905114e-02,  8.55998880e-02,  3.79637618e-02,  8.86462454e-03,
  2.25877163e-02,  1.50210728e-02,  1.04716486e-02,  2.13831683e-03,
  3.41282337e-02,  3.07630523e-02, -4.94275710e-03, -2.57485253e-01,
  7.02772941e-01,  5.42417752e-01,  4.82393408e-01,  8.49663532e-01,
  9.09941180e-01,  1.39132174e+00,  1.00000000e-01,  1.00000000e-01,
  1.00000000e-01,  1.00000000e-01,  1.00000000e-01,  1.00000000e-01,
  1.00000000e-01,  1.00000000e-01,  1.00000000e-01,  1.00000000e-01]

print(len(params))

40


In [38]:
# set number of assets which is equal to number of qubits
num_assets = 10
seed = 479  # To keep consistently generated data

p_vec = ParameterVector("theta", 40)
for i in range(len(params)):
    p_vec.params[i] = params[i]
    
SU2_linear = EfficientSU2(num_assets, entanglement='linear', reps=1)

# Generate expected return and covariance matrix from (random) time-series
stocks = [("TICKER%s" % i) for i in range(num_assets)]
data = RandomDataProvider( # Change to YahooDataProvider for real stock data
    tickers=stocks,
    start=datetime.datetime(2023, 1, 1),
    end=datetime.datetime(2023, 1, 30),
    seed=seed,
)

data.run()
ev = data.get_period_return_mean_vector() # Initial state to optimize over
covariance = data.get_period_return_covariance_matrix() # Hamiltonian Matrix

risk_factor = 0.5  # set risk factor
budget = 5  # set budget (number of assets to keep)

qp = PortfolioOptimization(expected_returns=ev, covariances=covariance, risk_factor=risk_factor, budget=budget).to_quadratic_program()
#qubo = converter.convert(qp) #constraintless format to pass as operator

## Setting up the Quantum Computer

In [22]:
# To get your own token make an account here: https://quantum.ibm.com
# To run on hardware with API token below only run once per kernel load
service = QiskitRuntimeService(channel="ibm_quantum", 
                               token="a6cee0a56416a409ee6af94b1cf3bb957c083d2bfd102c47416849a4455cd711626e8bba737191cf1882848c9ebe6d7e6ba17cda21c913546cec73323f71a95b")


In [23]:
# List real available quantum computer backends
service.backends(simulator=False, operational=True, min_num_qubits=25)

[<IBMBackend('ibm_brisbane')>,
 <IBMBackend('ibm_kyiv')>,
 <IBMBackend('ibm_sherbrooke')>]

In [24]:
# Choosing Brisbane as it is currently the least busy
real_device = service.backend('ibm_brisbane')

In [49]:
SU2_linear.measure_all()

# bound_SU2 = [SU2_linear.bind_parameters({p: val for val in values})]
SU2_linear.parameters[0][0] = p_vec.params[0]

TypeError: 'ParameterVectorElement' object does not support item assignment

In [37]:
real_device = service.backend('ibm_brisbane')
transpiled_SU2_linear = transpile(SU2_linear, real_device, optimization_level=0)
print("Depth of circuit is " + str(transpiled_SU2_linear.depth()))

Depth of circuit is 0


In [ ]:
job_vqe = real_device.run(transpiled_SU2_linear, shots=1000)
print('Job ID for VQE:', job_vqe.job_id())
job_id = job_vqe.job_id()

In [47]:
cobyla = COBYLA()
cobyla.set_options(maxiter=500)
svqe_mes = SamplingVQE(sampler=Sampler(), ansatz=SU2_full, optimizer=cobyla)
svqe = MinimumEigenOptimizer(svqe_mes)
result = svqe.solve(qp)

# Print the optimal portfolio based on budget and risk 
def print_result(result):
    selection = result.x
    value = result.fval
    print("Optimal: selection {}, value {:.4f}".format(selection, value))

    eigenstate = result.min_eigen_solver_result.eigenstate
    probabilities = (
        eigenstate.binary_probabilities()
        if isinstance(eigenstate, QuasiDistribution)
        else {k: np.abs(v) ** 2 for k, v in eigenstate.to_dict().items()}
    )

    # Prints all the options for portfolio and their vale
    print("\n----------------- Full result ---------------------")
    print("This is a list of all possible porfolio options and their valuation:")
    print("\n selection\t value \t\t probability")
    print("---------------------------------------------------")
    probabilities = sorted(probabilities.items(), key=lambda x: x[1], reverse=True)

    for k, v in probabilities:
        x = np.array([int(i) for i in list(reversed(k))])
        value = qp.objective.evaluate(x)
        print("%10s\t%.4f\t\t%.4f" % (x, value, v))


print_result(result)

ValueError: A backend or session must be specified.